### Import the Necessary Libraries for COVID Detection

In [3]:
#General Libraries for basic machine learning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import sklearn
import keras
import tensorflow

#Required Tensorflow models for the CNN Implementation
from tensorflow import keras
from keras.layers import Dense, Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten, MaxPooling2D, Dropout

from keras.applications import DenseNet121
from keras.applications.densenet import preprocess_input

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import img_to_array

from keras.models import Model
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

#Required sci-kit learn packages for working with dataset
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

#Libraries for working with system and mathematical operations
import os, random, math

#Working with warnings
import warnings
warnings.filterwarnings("ignore")

### Checking the Tensorflow version

In [4]:
print(tensorflow.__version__)

2.11.0
